*This notebook aims to create a matrix for logistic regression. It follows the 'all_demographic_no_timewindow.ipynb' notebook and will process SNOMED/Phecode mapping.*

process
1. load condition data
2. organize the source code (snomed - ICD, ICD - phecode)
3. 

In [ ]:
'''

To preprocess data for analysis

'''
import pandas as pd
import os
import numpy as np
import gc
import pickle

condition_data = pd.read_csv('condition_dataset.csv')

In [2]:
## select patients from age 18-90; return dataset_45023455_condition_df

# This query represents dataset "whole-cohort-all-condition" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_45023455_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (
                    SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (
                            SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                            WHERE
                                concept_id IN (
                                    137193, 137977, 3663207, 37311320, 4011630, 4021667, 4024705, 4027304, 4041287, 4042140, 40480457, 4070767, 4089228, 4094294, 4132559, 4167864, 4168498, 4172024, 4179304, 4196732, 4213737, 4274025, 4275722, 4297708, 43021226, 4303401, 432453, 433595, 437312, 437382
                                )       
                                AND full_text LIKE '%_rank1]%'      
                        ) a 
                            ON (
                                c.path LIKE CONCAT('%.',
                            a.id,
                            '.%') 
                            OR c.path LIKE CONCAT('%.',
                            a.id) 
                            OR c.path LIKE CONCAT(a.id,
                            '.%') 
                            OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1
                        )
                )  
                AND (
                    c_occurrence.PERSON_ID IN (
                        SELECT
                            distinct person_id  
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                        WHERE
                            cb_search_person.person_id IN (
                                SELECT
                                    person_id 
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                                WHERE
                                    age_at_consent BETWEEN 18 AND 90 
                            ) 
                        )
                )
            ) c_occurrence 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
                ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
                ON c_occurrence.condition_type_concept_id = c_type.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
                ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
                ON v.visit_concept_id = visit.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
                ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
        LEFT JOIN
            `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
                ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_45023455_condition_df = pd.read_gbq(
    dataset_45023455_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

#dataset_45023455_condition_df.head(5)

Downloading:   0%|          | 0/85408686 [00:00<?, ?rows/s]

In [3]:
## select patients from age 18-90; return dataset_45023455_person_df
# This query represents dataset "whole-cohort-all-condition" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_45023455_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (
            SELECT
                distinct person_id  
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (
                    SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        age_at_consent BETWEEN 18 AND 90 
                ) 
            )"""

dataset_45023455_person_df = pd.read_gbq(
    dataset_45023455_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

#dataset_45023455_person_df.head(5)


Downloading:   0%|          | 0/412573 [00:00<?, ?rows/s]

In [4]:
dataset_45023455_condition_df['person_id'].nunique()

253861

In [5]:
dataset_45023455_person_df['person_id'].nunique()

412573

### Load mapping dictionary file

In [5]:
# load mapping dictionary: snomed2icd10
snomed2icd = pd.read_csv('snomed_icd_mapping.csv',dtype={'SNOMED': str, 'ICD10':str})
snomed2icd = snomed2icd.rename(columns ={'SNOMED':'source_concept_code','ICD10CM':'ICDcode'})


In [ ]:
'''
Map the SNOMED to ICD-10-CM
'''
# Load SNOMED to ICD mapping table 
snomed2icd = pd.read_csv('snomed_icd_mapping.csv',dtype={'SNOMED': str, 'ICD10':str})
snomed2icd = snomed2icd.rename(columns ={'SNOMED':'source_concept_code',
                                         'ICD10CM':'ICDcode'}) # Ensure column name is matching

# Select interested columns
df_condition_all = condition_data[['person_id','standard_concept_code','source_concept_code','source_vocabulary','condition_start_datetime']]

# Subset records that already use ICD codes
df_condition_icd = df_condition_all.loc[df_condition_all['source_vocabulary'].isin(['ICD10CM', 'ICD9CM'])]

# Subset records that use SNOMED codes
df_condition_snomed = df_condition_all.loc[dataset_condition_df['source_vocabulary'] == 'SNOMED' ]
df_condition_snomed = df_condition_snomed.reset_index(drop = True)

# Map SNOMED codes to ICD using the reference table
df_condition_snomed_mapped = df_condition_snomed.merge(snomed2icd, how='left', on = 'source_concept_code')

# Remove records with no valid ICD mapping
df_condition_snomed_mapped = df_condition_snomed_mapped.dropna(subset=['ICDcode'])

# Standardize ICD column format for both sources
df_condition_icd.loc[:, 'ICDcode'] = df_condition_icd['source_concept_code']
df_condition_icd.loc[:, 'ICDsource'] = df_condition_icd['source_vocabulary']

# Concatenate both sources into a unified ICD-coded dataset
df_condition_with_icd= pd.concat([df_condition_icd,df_condition_snomed_mapped],ignore_index=True)
df_condition_with_icd.head()

In [ ]:
'''
Map ICD to Phecode
'''
# Load ICD to Phecode mapping table
icd9phe = pd.read_csv('ICD_9CM_phecode.csv',dtype = {'Phecode':str, 'ICD':str})
icd10phe = pd.read_csv('ICD_10CM_phecode.csv',dtype = {'Phecode':str, 'ICD':str})

icd9phe = icd9phe.rename(columns = {'ICD':'ICDcode'})

icd10phe = icd10phe.rename(columns = {'ICD':'ICDcode'})


dataset_condition_concat_icd9_df = df_condition_with_icd[df_condition_with_icd['source_vocabulary'] == 'ICD9CM']
dataset_condition_concat_icd10_df = df_condition_with_icd[(df_condition_with_icd['source_vocabulary'] == 'ICD10CM') |(df_condition_with_icd['source_vocabulary'] == 'SNOMED')]

phe_icd9_df = dataset_condition_concat_icd9_df.merge(icd9phe, how = 'left',on='ICDcode')
phe_icd10_df = dataset_condition_concat_icd10_df.merge(icd10phe, how = 'left',on='ICDcode')
phe_condition_df = pd.concat([phe_icd9_df,phe_icd10_df])
non_mapping_code = phe_condition_df[phe_condition_df['Phecode'].isna()]
non_mapping_code = non_mapping_code[['ICDcode','ICDsource']].drop_duplicates()

# check_list = non_mapping_code[non_mapping_code['ICDsource'] == 'ICD9CM'][['ICDcode']]
# check_list = non_mapping_code[non_mapping_code['ICDsource'] == 'ICD10CM'][['ICDcode']]
# check_list = non_mapping_code[non_mapping_code['ICDsource'] == 'ICD10CM_Mapping'][['ICDcode']]
# icd2phe[icd2phe['source_concept_code'].isin(check_list)]

phe_condition_drop_df = phe_condition_df.dropna(subset=['Phecode'])
phe_condition_drop_df = phe_condition_drop_df.drop_duplicates()

print ('number of phecode')
all_phecode= set(phe_condition_drop_df['Phecode'])
len(all_phecode)#1847 phecode => 1848

In [ ]:
'''
Map ICD codes to Phecodes
'''

# Load ICD-to-Phecode mapping tables
df_icd9_to_phe = pd.read_csv('ICD_9CM_phecode.csv', dtype={'Phecode': str, 'ICD': str})
df_icd10_to_phe = pd.read_csv('ICD_10CM_phecode.csv', dtype={'Phecode': str, 'ICD': str})

# Standardize column names
df_icd9_to_phe = df_icd9_to_phe.rename(columns={'ICD': 'ICDcode'})
df_icd10_to_phe = df_icd10_to_phe.rename(columns={'ICD': 'ICDcode'})

# Subset ICD9-coded conditions
df_condition_icd9 = df_condition_with_icd[
    df_condition_with_icd['source_vocabulary'] == 'ICD9CM'
]

# Subset ICD10-coded conditions, including SNOMED-mapped
df_condition_icd10 = df_condition_with_icd[
    (df_condition_with_icd['source_vocabulary'] == 'ICD10CM') |
    (df_condition_with_icd['source_vocabulary'] == 'SNOMED')
]
# Map ICD-9-CM codes to Phecodes
df_phe_icd9 = df_condition_icd9.merge(df_icd9_to_phe, how='left', on='ICDcode')

# Map ICD-10-CM codes to Phecodes
df_phe_icd10 = df_condition_icd10.merge(df_icd10_to_phe, how='left', on='ICDcode')

# Concatenate all Phecode-mapped condition records
df_phe_condition = pd.concat([df_phe_icd9, df_phe_icd10])

# Identify unmapped ICD codes
df_unmapped_icd = df_phe_condition[df_phe_condition['Phecode'].isna()]
df_unmapped_icd = df_unmapped_icd[['ICDcode', 'ICDsource']].drop_duplicates()
print('ICD codes not mapped to any Phecode:', df_unmapped_icd.ICDcode.unique())

# Remove unmapped records and duplicates
df_phe_condition_mapped = df_phe_condition.dropna(subset=['Phecode'])
df_phe_condition_mapped = df_phe_condition_mapped.drop_duplicates()

print('Number of unique Phecodes:',len(set(df_phe_condition_mapped['Phecode'])))

In [ ]:
'''
Identify HS case and clean control groups
'''

# --- Define HS-related codes ---
HS_SNOMED_list = ['69741000', '59393003', '402826001', '402828000', '18638007']
HS_ICD_code = ['705.83', 'L73.2']
white_list = [
    "724834006", "1254916008", "275448003", "59393003", "18638007",
    "404176003", "402825002", "238992001", "403640008", "69741000",
    "402827005", "785724007", "238754005", "402826001", "1254915007",
    "402828000", "402706007", "404175004", "402752000", "74578003"
]

# --- Identify HS case person_ids ---
df_hs_pid = df_phe_condition_mapped[
    df_phe_condition_mapped['standard_concept_code'].isin(HS_SNOMED_list) |
    df_phe_condition_mapped['ICDcode'].isin(HS_ICD_code)
][['person_id']].drop_duplicates()

# --- Identify control person_ids: not in HS group ---
df_control_condition = df_phe_condition_mapped[
    ~df_phe_condition_mapped['person_id'].isin(df_hs_pid['person_id'])
]

# --- Remove control individuals who appear in white-listed SNOMED codes ---
df_white_list_pid = df_control_condition[df_control_condition['standard_concept_code'].isin(white_list)]['person_id'].unique()
df_control_clean = df_control_condition[~df_control_condition['person_id'].isin(df_white_list_pid)]

# --- Final unique IDs ---
print('Number of case individuals:', len(df_hs_pid))
print('Number of control individuals:', df_control_clean['person_id'].nunique())

# --- Save PIDs
df_hs_pid.to_csv('HS_pid.csv', index=False)
df_control_clean_pid = df_control_clean[['person_id']].drop_duplicates()
df_control_clean_pid.to_csv('control_pid.csv', index=False)

# --- Select cohort
df_cohort = df_phe_condition_mapped[
    df_phe_condition_mapped['person_id'].isin(df_hs_pid['person_id']) |
    df_phe_condition_mapped['person_id'].isin(df_control_clean['person_id'])
]
df_cohort['HS'] = None
df_cohort.loc[df_cohort['person_id'].isin(df_hs_pid['person_id']), 'HS'] = 1
df_cohort.loc[df_cohort['person_id'].isin(df_control_clean['person_id']), 'HS'] = 0

In [ ]:
# back up

In [ ]:
'''
Identify case and control groupㄋ
'''
# Define HS case codes
HS_SNOWMED_list=['69741000','59393003','402826001','402828000','18638007']
HS_ICD_code = ['705.83','L73.2']

# Define white-listed SNOMED codes
white_list = [
    "724834006", "1254916008", "275448003", "59393003", "18638007",
    "404176003", "402825002", "238992001", "403640008", "69741000",
    "402827005", "785724007", "238754005", "402826001", "1254915007",
    "402828000", "402706007", "404175004", "402752000", "74578003"
]
# Identify HS cases by SNOMED code
df_hs_condition_snomed = df_phe_condition_mapped[
    df_phe_condition_mapped['standard_concept_code'].isin(HS_SNOMED_list)
]
# Identify HS cases by ICD code
df_hs_condition_icd = df_phe_condition_mapped[
    df_phe_condition_mapped['ICDcode'].isin(HS_ICD_code)
]
# Extract unique person IDs for HS cases
df_hs_pid_snomed = df_hs_condition_snomed[['person_id']].drop_duplicates()
df_hs_pid_icd = df_hs_condition_icd[['person_id']].drop_duplicates()

# Combine SNOMED-based and ICD-based HS cases
df_hs_pid = pd.concat([df_hs_pid_icd, df_hs_pid_snomed]).drop_duplicates().reset_index(drop=True)
hs_pid_list = df_hs_pid['person_id']

# Identify control group: individuals without any HS code
df_control_condition = df_phe_condition_mapped[
    ~df_phe_condition_mapped['person_id'].isin(hs_pid_list)
]
df_control_pid = df_control_condition[['person_id']].drop_duplicates()

print('Number of case individuals:' , len(df_hs_pid))
print('Number of control individuals:', len(df_control_pid))

# Remove control pid within whitelist

df_white_list_condition = df_control_condition[df_control_condition['standard_concept_code'].isin(white_list)]
white_list_pid = df_white_list_condition.person_id.drop_duplicates()
df_control_clean = df_control_condition[~df_control_condition['person_id'].isin(white_list_pid)]

In [ ]:
'''
Create Phecode Matrix
'''
def create_matrix(df):
    # Drop duplicate entries for person_id and Phecode
    df = df.drop_duplicates(subset=['person_id', 'Phecode'])
    
    # Create the binary matrix using pivot_table
    binary_matrix = df.assign(value=1).pivot_table(
        index='person_id',columns='Phecode', values='value', fill_value=0)
        
    return binary_matrix

phecode_matrix = create_matrix(phe_condition_select)
df_demo = demographic_data[[
    'person_id','sex_at_birth','date_of_birth','race','ethnicity'
]].drop_duplicates()

phecode_matrix_demo = phecode_matrix.merge(df_demo, on = 'person_id', how = 'left')
phecode_matrix_demo = phecode_matrix_demo.merge(df_cohort[['person_id', 'HS']], on='person_id', how='left')

In [47]:
# # only keep code related columns to reduce memory
# dataset_condition_df = dataset_45023455_condition_df[['person_id','standard_concept_code','source_concept_code','source_vocabulary','condition_start_datetime']]
# # select subset that have ICD code
#dataset_condition_ICD_df = dataset_condition_df.loc[dataset_condition_df['source_vocabulary'].isin(['ICD10CM', 'ICD9CM'])]
# select subset that have SNOMED code
dataset_condition_nonICD_df_1 = dataset_condition_df.loc[dataset_condition_df['source_vocabulary'] == 'SNOMED' ]
dataset_condition_nonICD_df_1 = dataset_condition_nonICD_df_1.reset_index(drop = True)
#dataset_condition_nonICD_df = dataset_condition_nonICD_df.set_index('standard_concept_code')

In [48]:
dataset_condition_nonICD_df_1.shape #(6290946, 19)

(6290946, 5)

In [49]:
snomed2icd_1['ICDsource'] = 'ICD10CM_Mapping_1'
snomed2icd_1.head(5)

,source_concept_code,ICDcode,effectiveTime,ICDsource
0,127009,O03.84,20170901,ICD10CM_Mapping_1
1,134006,L67.8,20170901,ICD10CM_Mapping_1
2,150003,R32,20170901,ICD10CM_Mapping_1
3,151004,A54.81,20170901,ICD10CM_Mapping_1
4,162004,F31.13,20170901,ICD10CM_Mapping_1


In [50]:
merge_condition_nonICD_1 = dataset_condition_nonICD_df_1.merge(snomed2icd_1, how='left', on = 'source_concept_code')

In [51]:
merge_condition_nonICD_1.head(5)

,person_id,standard_concept_code,source_concept_code,source_vocabulary,condition_start_datetime,ICDcode,effectiveTime,ICDsource
0,1247519,285058000,285058000,SNOMED,2020-10-20 00:00:00+00:00,NaN,NaN,NaN
1,1600303,285058000,285058000,SNOMED,2017-03-16 00:00:00+00:00,NaN,NaN,NaN
2,1668080,285058000,285058000,SNOMED,2017-07-11 00:00:00+00:00,NaN,NaN,NaN
3,3008258,285058000,285058000,SNOMED,2018-12-05 00:00:00+00:00,NaN,NaN,NaN
4,1077108,285058000,285058000,SNOMED,2013-06-14 00:00:00+00:00,NaN,NaN,NaN


In [52]:
# The nan SNOMED-ICD mapping
merge_condition_nonICD_1[merge_condition_nonICD_1['ICDcode'].isna()].nunique()

person_id                   29705
standard_concept_code       10612
source_concept_code         10849
source_vocabulary               1
condition_start_datetime    75451
ICDcode                         0
effectiveTime                   0
ICDsource                       0
dtype: int64

In [19]:
# The nan SNOMED-ICD mapping
# merge_condition_nonICD[merge_condition_nonICD['ICDcode'].isna()].nunique()

person_id                   28673
standard_concept_code        5640
source_concept_code          5714
source_vocabulary               1
condition_start_datetime    59283
ICDcode                         0
ICDsource                       0
dtype: int64

In [53]:
merge_condition_nonICD_1[~merge_condition_nonICD_1['source_concept_code'].isna()].nunique()

person_id                    33334
standard_concept_code        25475
source_concept_code          26006
source_vocabulary                1
condition_start_datetime    119786
ICDcode                       7781
effectiveTime                   13
ICDsource                        1
dtype: int64

In [20]:
merge_condition_nonICD[~merge_condition_nonICD['source_concept_code'].isna()].nunique()

person_id                    33334
standard_concept_code        25475
source_concept_code          26006
source_vocabulary                1
condition_start_datetime    119786
ICDcode                       9684
ICDsource                        1
dtype: int64

#### Check codes that cannot find a ICD mapping

In [55]:
na_rows_1 = merge_condition_nonICD_1[merge_condition_nonICD_1['ICDcode'].isna()]
na_rows_1.shape#570386 records have NA ==> 1774239
len(set(na_rows_1['standard_concept_code']))#9618 SNOMED codes cannot find a mapping ==> 5640

10612

#### Remove unmapped records

In [56]:
merge_condition_nonICD_dropna_1 = merge_condition_nonICD_1.dropna(subset=['ICDcode'])

In [57]:
merge_condition_nonICD_dropna_1.shape #(6681125, 7)

(3680300, 8)

In [58]:
merge_condition_nonICD_dropna_1.shape #(4906886, 7、

(3680300, 8)

#### Concat

In [60]:
## merge mapped df to original df
dataset_condition_concat_df_1 = pd.concat([dataset_condition_ICD_df,merge_condition_nonICD_dropna_1],ignore_index=True)
dataset_condition_concat_df_1

,person_id,standard_concept_code,source_concept_code,source_vocabulary,condition_start_datetime,ICDcode,ICDsource,effectiveTime
0,4567145,59621000,I10,ICD10CM,2020-12-13 13:54:36+00:00,I10,ICD10CM,NaN
1,7189522,195967001,493.90,ICD9CM,2013-09-05 07:35:00+00:00,493.90,ICD9CM,NaN
2,3061977,737303004,Z90.49,ICD10CM,2019-08-15 06:00:00+00:00,Z90.49,ICD10CM,NaN
3,2097491,40739000,R13.10,ICD10CM,2021-10-16 06:00:00+00:00,R13.10,ICD10CM,NaN
4,1084350,709111008,J96.01,ICD10CM,2022-05-02 06:00:00+00:00,J96.01,ICD10CM,NaN
...,...,...,...,...,...,...,...,...
82676903,1093747,77386006,77386006,SNOMED,2020-12-08 23:59:00+00:00,Z33.1,ICD10CM_Mapping_1,20170901.0
82676904,1663024,77386006,77386006,SNOMED,2019-12-04 14:28:00+00:00,Z33.1,ICD10CM_Mapping_1,20170901.0
82676905,1769202,77386006,77386006,SNOMED,2018-04-19 15:32:00+00:00,Z33.1,ICD10CM_Mapping_1,20170901.0
82676906,1400989,77386006,77386006,SNOMED,2019-03-04 07:51:00+00:00,Z33.1,ICD10CM_Mapping_1,20170901.0


#### Map to ICD

In [61]:
dataset_condition_concat_icd9_df_1 = dataset_condition_concat_df_1[dataset_condition_concat_df_1['source_vocabulary'] == 'ICD9CM']

In [62]:
dataset_condition_concat_icd10_df_1 = dataset_condition_concat_df_1[(dataset_condition_concat_df_1['source_vocabulary'] == 'ICD10CM') |(dataset_condition_concat_df_1['source_vocabulary'] == 'SNOMED')]

In [63]:
print(dataset_condition_concat_icd9_df_1.shape)#(28996423, 7)
print(dataset_condition_concat_icd10_df_1.shape)
print(dataset_condition_concat_df_1.shape)

(28996423, 8)
(53680485, 8)
(82676908, 8)


In [66]:
icd9phe = icd9phe.rename(columns = {'ICD':'ICDcode'})
phe_icd9_df_1 = dataset_condition_concat_icd9_df_1.merge(icd9phe, how = 'left',on='ICDcode')

In [69]:
icd10phe = icd10phe.rename(columns = {'ICD':'ICDcode'})
phe_icd10_df_1 = dataset_condition_concat_icd9_df_1.merge(icd10phe, how = 'left',on='ICDcode')

In [71]:
phe_condition_df_1 = pd.concat([phe_icd9_df_1,phe_icd10_df_1])

In [72]:
non_mapping_code_1 = phe_condition_df_1[phe_condition_df_1['Phecode'].isna()]
non_mapping_code_1 = non_mapping_code_1[['ICDcode','ICDsource']].drop_duplicates()

In [73]:
non_mapping_code['ICDsource'].value_counts()

ICDsource
ICD10CM            1083
ICD9CM              370
ICD10CM_Mapping     274
Name: count, dtype: int64

#### Remove NA Phecode mapping 

In [74]:
# remove NA
phe_condition_drop_df_1 = phe_condition_df_1.dropna(subset=['Phecode'])
phe_condition_drop_df_1 = phe_condition_drop_df_1.drop_duplicates()

In [75]:
phe_condition_drop_df_1.shape #(24085331, 11)

(24085331, 11)

In [76]:
all_phecode_1= set(phe_condition_drop_df_1['Phecode'])
len(all_phecode_1)#1847 phecode => 1829

1829

In [4]:

match_pid = pd.read_csv('result_v2/matched_person_ids.csv')

In [7]:
match_pid

,x
0,2395521
1,2563639
2,1734015
3,2010154
4,1067462
...,...
24096,1325844
24097,1449547
24098,1255958
24099,3078108


In [3]:
hs_pid = pd.read_csv('result_v2/HS_pid_v2.csv')

In [12]:
hs_pid

,Unnamed: 0,person_id
0,0,1184743
1,1,1630386
2,2,1749834
3,3,1483576
4,4,3122578
...,...,...
2252,2252,1275562
2253,2253,1792421
2254,2254,2375465
2255,2255,1993406


In [8]:
all_pid = pd.concat([hs_pid,control_pid_final_old])

In [10]:
all_pid = all_pid['person_id']
check = match_pid[~match_pid['x'].isin(all_pid)]
